In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import h5py
from arts_preprocess_utils import load_dataset
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# !!! remember to clear session/graph if you rebuild your graph to avoid out-of-memory errors !!!
def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s

In [ ]:
# remember to clear session if you start building graph from scratch!
s = reset_tf_session()
# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)

## Prepare for training

In [ ]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig = load_dataset('/root/work/datasets/wikiart_mini_ic.h5')

In [ ]:
train_set_x_orig.shape

In [ ]:
test_set_x_orig.shape

In [ ]:
img_Height = train_set_x_orig.shape[1]
img_Width = train_set_x_orig.shape[2]
N_CLASSES= len(np.unique(test_set_y_orig))-1

**Plot painting example**

In [ ]:
plt.imshow(np.clip(train_set_x_orig[7], 0, 255).astype('uint8')[...,::-1])

In [ ]:
train_set_y_orig[7]

In [ ]:
X_train = train_set_x_orig
y_train = train_set_y_orig

X_test = test_set_x_orig
y_test = test_set_y_orig

## Split train dev

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(train_set_x_orig, train_set_y_orig, test_size=0.25)

## Training

**Simple model**

In [ ]:
from models import Simple_Model

simple_model = Simple_Model()
model = simple_model.get_model(input_shape = (img_Height,img_Width,3), n_classes=N_CLASSES)

In [ ]:
model.summary()

**Inception model**

In [ ]:
from models import Inception_model

inception_model = Inception_model()
model = inception_model.get_model(input_shape =(img_Height,img_Width,3), n_classes=N_CLASSES, use_imagenet=True)

In [ ]:
model.summary()

**ResNet50**

In [ ]:
from models import ResNet_model

resnet_model = ResNet_model()
model = resnet_model.get_model(input_shape =(img_Height,img_Width,3), n_classes=N_CLASSES, use_imagenet=True)

In [ ]:
model.summary()

**set all layers trainable by default and prepare batch norm for fine-tuning**

In [ ]:
# set all layers trainable by default and prepare batch norm for fine-tuning
for layer in model.layers:
    layer.trainable = True
    if isinstance(layer, keras.layers.BatchNormalization):
        # we do aggressive exponential smoothing of batch norm 
        # parameters to faster adjust to our new dataset
        layer.momentum = 0.8
    
# fix deep layers (fine-tuning only last n)
for layer in model.layers[:-10]:
    layer.trainable = False

### Normalize data

In [ ]:
X_train = resnet_model.preprocess_input(X_train)
X_dev = resnet_model.preprocess_input(X_dev)


#X_train = inception_model.preprocess_input(X_train)
#X_dev = inception_model.preprocess_input(X_dev)

In [ ]:
batch_size = 32

# this is the augmentation configuration we will use for training:
train_datagen = ImageDataGenerator(featurewise_center= True, 
                                   featurewise_std_normalization=True, 
                                   shear_range=0.2, 
                                   zoom_range=0.2)
# this is the augmentation configuration we will use for testing:
# only rescaling
dev_datagen = ImageDataGenerator()

In [ ]:
train_datagen.fit(X_train)
dev_datagen.fit(X_dev)

In [ ]:
y_train = keras.utils.np_utils.to_categorical(num_classes=N_CLASSES, y= train_set_y_orig)
y_test = keras.utils.np_utils.to_categorical(num_classes=N_CLASSES, y= test_set_y_orig)

**Train model**

In [ ]:
# compile new model
model.compile(
    loss='binary_crossentropy',  
    optimizer=keras.optimizers.adamax(lr=1e-4),  # we can take big lr here because we fixed first layers
    metrics=['accuracy']  # report accuracy during training
)

In [ ]:
model.fit_generator(
        train_datagen.flow(x=X_train,y=y_train, batch_size=batch_size),
        steps_per_epoch=len(X_train) // batch_size,
        epochs=20,
        validation_data=dev_datagen.flow(x=X_dev, y= y_dev, batch_size=batch_size),
        validation_steps=len(X_dev) // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

In [ ]:
model.save('inception_impressionism.h5')

### Test model

**Test generator**

In [ ]:
X_test = resnet_model.preprocess_input(X_test)

#X_test = inception_model.preprocess_input(X_test)

In [ ]:
# only rescaling
test_datagen = ImageDataGenerator()
test_datagen.fit(X_test)

In [ ]:
model.evaluate_generator(test_datagen.flow(x=X_test,y=y_test, batch_size=batch_size),
                        steps=len(X_test))

In [ ]:
plt.imshow(np.clip(X_dev[18], 0, 255).astype('uint8')[...,::-1])

In [ ]:
X = X_dev[10:20,:,:,:] * (1./255)

In [ ]:
model.predict(X).round()

In [ ]:
y_dev[10:20]

### MAE

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
pred = model.predict(X_test)

In [ ]:
mean_absolute_error(y_pred=pred, y_true=y_dev)

## Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred=pred.round(), y_true=y_test)